# 03 Instrumental Variables (2SLS)

Endogeneity, instruments, and two-stage least squares (2SLS).


## Table of Contents
- [Simulate endogeneity](#simulate-endogeneity)
- [OLS vs 2SLS](#ols-vs-2sls)
- [First-stage + weak IV checks](#first-stage-weak-iv-checks)
- [Interpretation + limitations](#interpretation-limitations)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Causal notebooks focus on **identification**: what would have to be true for a coefficient to represent a causal effect.
You will practice:
- building a county-year panel,
- fixed effects (TWFE),
- clustered standard errors,
- DiD + event studies,
- IV/2SLS.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating regression output as causal without stating identification assumptions.
- Using non-clustered SE when shocks are correlated within groups (e.g., states).

## Matching Guide
- `docs/guides/07_causal/03_instrumental_variables_2sls.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/07_causal/03_instrumental_variables_2sls.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Practice IV/2SLS by simulating a classic endogeneity problem.

We do this synthetically so you can see the bias and how IV can fix it under assumptions.



## Primer: Panel and IV regression with `linearmodels`

`statsmodels` is great for OLS inference, but panel and IV workflows are often cleaner with `linearmodels`.

This project uses `linearmodels` for:
- **PanelOLS** (fixed effects / TWFE)
- **IV2SLS** (instrumental variables)

### Panel data shape
Most panel estimators expect a **MultiIndex**:
- level 0: entity (e.g., county `fips`)
- level 1: time (e.g., `year`)

In pandas:
```python
# df is a DataFrame with columns fips, year, y, x1, x2
# df = df.set_index(['fips', 'year']).sort_index()
```

### Minimal PanelOLS (two-way fixed effects)
```python
from linearmodels.panel import PanelOLS

# y: Series with MultiIndex
# X: DataFrame with MultiIndex

# model: y_it = beta'X_it + alpha_i + gamma_t + eps_it
# res = PanelOLS(y, X, entity_effects=True, time_effects=True).fit(cov_type='robust')
# print(res.summary)
```

### Clustered standard errors (common in applied work)
If errors are correlated within clusters (e.g., state), use clustered SE:
```python
# clusters must align with y/X index (same rows)
# res = PanelOLS(y, X, entity_effects=True, time_effects=True).fit(
#     cov_type='clustered',
#     clusters=df['state'],  # e.g., state-level clustering
# )
```

### Minimal IV2SLS (one endogenous regressor)
```python
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

# y: Series
# endog: DataFrame with endogenous regressor(s)
# exog: DataFrame with controls (include a constant)
# instr: DataFrame with instruments

# exog = sm.add_constant(exog, has_constant='add')
# res = IV2SLS(y, exog, endog, instr).fit(cov_type='robust')
# print(res.summary)
```

### Practical rule
- If the goal is **causal identification**, always write down the assumptions first (parallel trends, exclusion restriction, etc.).
- Then treat the model output as conditional on those assumptions, not as “truth”.


<a id="simulate-endogeneity"></a>
## Simulate endogeneity

### Goal
Create data where:
- x is correlated with the error term (endogenous)
- z shifts x but not y directly (instrument)



### Your Turn: Simulate (y, x, z)


In [ ]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
n = 2000

# Instrument
z = rng.normal(size=n)

# Hidden confounder
u = rng.normal(size=n)

# Endogenous regressor: depends on z and u
x = 0.8*z + 0.8*u + rng.normal(size=n)

# Error term correlated with u
eps = 0.8*u + rng.normal(size=n)

beta_true = 1.5
y = beta_true * x + eps

df = pd.DataFrame({'y': y, 'x': x, 'z': z})
df.head()



<a id="ols-vs-2sls"></a>
## OLS vs 2SLS

### Goal
Compare naive OLS (biased) to IV/2SLS.



### Your Turn: Fit OLS and 2SLS


In [ ]:
import statsmodels.api as sm
from src.causal import fit_iv_2sls

# OLS
ols = sm.OLS(df['y'], sm.add_constant(df[['x']], has_constant='add')).fit()
print('OLS beta:', float(ols.params['x']))

# 2SLS
iv = fit_iv_2sls(df, y_col='y', x_endog='x', x_exog=[], z_cols=['z'])
print('IV beta :', float(iv.params['x']))

iv.summary



<a id="first-stage-weak-iv-checks"></a>
## First-stage + weak IV checks

### Goal
Inspect the first stage and discuss instrument strength.



### Your Turn: Inspect first stage


In [ ]:
# TODO: Explore first-stage outputs.
# Hint: `iv.first_stage` is usually informative.
iv.first_stage



<a id="interpretation-limitations"></a>
## Interpretation + limitations

Write 5-8 sentences on:
- relevance and exclusion in this synthetic setup
- what would break IV in real data
- why IV identifies a local effect when effects are heterogeneous (LATE intuition)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected output: (see notebook front matter)
# TODO: If you created a panel DataFrame, verify the indexing + core columns.
# Example (adjust variable names):
# assert isinstance(panel.index, pd.MultiIndex)
# assert panel.index.names[:2] == ['fips', 'year']
# assert panel['year'].astype(int).between(1900, 2100).all()
# assert panel['fips'].astype(str).str.len().eq(5).all()
#
# TODO: Write 2-3 sentences:
# - What is the identification assumption for your causal estimate?
# - What diagnostic/falsification did you run?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Simulate endogeneity</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_instrumental_variables_2sls — Simulate endogeneity
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
n = 2000
z = rng.normal(size=n)          # instrument
u = rng.normal(size=n)          # unobserved confounder

x = 0.8*z + 0.8*u + rng.normal(size=n)  # endogenous regressor
eps = 0.8*u + rng.normal(size=n)        # error correlated with x

beta_true = 1.5
y = beta_true * x + eps

df = pd.DataFrame({'y': y, 'x': x, 'z': z})
df.head()
```

</details>

<details><summary>Solution: OLS vs 2SLS</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_instrumental_variables_2sls — OLS vs 2SLS
import statsmodels.api as sm
from src.causal import fit_iv_2sls

ols = sm.OLS(df['y'], sm.add_constant(df[['x']], has_constant='add')).fit()
print('OLS beta:', float(ols.params['x']))

iv = fit_iv_2sls(df, y_col='y', x_endog='x', x_exog=[], z_cols=['z'])
print('IV beta :', float(iv.params['x']))
```

</details>

<details><summary>Solution: First-stage + weak IV checks</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_instrumental_variables_2sls — First-stage + weak IV checks
# Inspect first stage output (instrument strength):
# iv.first_stage
```

</details>

<details><summary>Solution: Interpretation + limitations</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_instrumental_variables_2sls — Interpretation + limitations
# Write 3-5 sentences on:
# - relevance + exclusion in your simulated setup
# - why IV can fix endogeneity here
```

</details>

